# Clean up typos and standardize column values for merging of datasets

In [37]:
import pandas as pd
import pickle
import numpy as np

In [81]:
maln_survey='Ethiopia/original_data/ethiopia-encu-smart-malnutrition-data-2000-2019.csv'
ndvi_fname='Ethiopia/original_data/ethiopia_ndvi_2002-2019.csv'
chirp_fname='Ethiopia/original_data/ethiopia_chirp_2002-2019.csv'
pop_fname='Ethiopia/original_data/ethiopia-population-projection-at-admin-2-level-2000-2020-normalized.csv' 
crop_df='Ethiopia/original_data/ethiopia-crop-production-statistics-meher-and-belg-admin-2-level-2004-2017.csv'


In [82]:
maln_df=pd.read_csv(maln_survey, encoding = "ISO-8859-1")
ndvi_df=pd.read_csv(ndvi_fname)
pop_df=pd.read_csv(pop_fname)
crop_df=pd.read_csv(crop_df)
chirp_df_std=pd.read_csv(chirp_fname)
cpi_df_std=pd.read_csv('Ethiopia/cpi/ethiopia-cpi-2007-2019-country-level.csv')
consumption_df=pd.read_csv('Ethiopia/consumption_exp/Ethiopia_household_expenditure_daily.csv')

In [83]:
def eth_pop_admin2(df, merge_col_names):
    """
    This function selects the columns of the normalized population dataset and melts it down
    to long format. The population value is extracted from column headings staring with 'BTOTL_{year}'
    df: input dataframe
    merge_col_names: list of columns for melting dataframe (e.g., ['admin1', 'admin2']).
    """
    Btotal_cols= [col for col in df.columns if 'BTOTL' in col]
    select_cols=[
        'COUNTRY',
         merge_col_names[0],       
         merge_col_names[1]]+Btotal_cols
    pop_btotal=df[select_cols]
    pop_melt=pd.melt(pop_btotal, id_vars=['COUNTRY']+merge_col_names, 
                 value_vars=Btotal_cols,value_name='POP')
    
    # new data columns with split value columns 
    new = pop_melt["variable"].str.split("_", n = 1, expand = True) 
    pop_melt['YEAR']=new[1]
    pop_melt.drop(columns=['variable'], inplace=True)
    pop_melt.rename(columns={'COUNTRY': 'admin0'}, inplace=True)
    
    return pop_melt

In [84]:
pop_df_melt=eth_pop_admin2(pop_df, ['admin1', 'admin2'])
pop_df_melt.head()

,admin0,admin1,admin2,POP,YEAR
0,Ethiopia,Addis Ababa,Addis Ketema,"261,024",2000
1,Ethiopia,Addis Ababa,Akaki Kaliti,"137,381",2000
2,Ethiopia,Addis Ababa,Arada,"253,286",2000
3,Ethiopia,Addis Ababa,Bole,"219,404",2000
4,Ethiopia,Addis Ababa,Gulele,"274,811",2000


In [42]:
pop_df.head()

,COUNTRY,admin1,admin2,BTOTL_2000,B0004_2000,B0509_2000,B1014_2000,B1519_2000,B2024_2000,B2529_2000,...,F3539_2020,F4044_2020,F4549_2020,F5054_2020,F5559_2020,F6064_2020,F6569_2020,F7074_2020,F7579_2020,F80PL_2020
0,Ethiopia,Addis Ababa,Addis Ketema,"261,024","17,790","15,844","21,287","37,325","42,166","34,177",...,"8,402","5,992","6,356","5,394","5,196","3,562","3,133","2,153","1,379","1,006"
1,Ethiopia,Addis Ababa,Akaki Kaliti,"137,381","13,705","10,937","12,522","17,062","19,303","16,469",...,"11,841","7,756","7,739","5,526","5,040","2,889","2,577","1,670","1,115",853
2,Ethiopia,Addis Ababa,Arada,"253,286","16,752","14,242","18,968","32,513","39,593","33,388",...,"6,142","4,430","4,663","3,525","3,250","2,161","1,912","1,320",902,672
3,Ethiopia,Addis Ababa,Bole,"219,404","21,331","15,645","17,060","27,474","34,504","29,246",...,"22,365","14,873","13,304","9,686","8,637","4,711","4,450","2,764","1,890","1,459"
4,Ethiopia,Addis Ababa,Gulele,"274,811","23,328","18,032","22,007","34,508","41,786","36,488",...,"9,516","6,124","6,309","5,074","5,013","3,262","2,907","1,934","1,237",893


In [43]:
typo_file = open("Ethiopia/eth_typo.pkl",'rb')
typo_dict = pickle.load(typo_file)
typo_file.close()


In [8]:
maln_df.head()

,Admin1,Admin2,Admin3,Month in survey,year,Cluster size,Sample size,GAM Z-scores,GAM CI,SAM Z-scores,...,Morbidity,Major illnesses/symptoms,Measles vaccination (card only),Measles vaccination95 CI,Measles vaccination (card + recall),Measles vaccinationcard + recall 95 CI,Vitamin A suplement,Vitamin A 95 CI,BCG coverage,BCG coverage CI
0,Afar,Zone 01,Afambo,August,2002,NaN,NaN,27.7,26.4- 29,3.80,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Afar,Zone 01,Afambo,February,2003,NaN,NaN,9.9,NaN,0.30,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Afar,Zone 01,Asayta,January,2006,NaN,NaN,11.1,8.4-14.4,1.30,...,0.159,NaN,NaN,NaN,NaN,NaN,0.905,NaN,NaN,NaN
3,Afar,Zone 01,Asayta,November,2003,NaN,NaN,13.8,10.8  17.4,1.80,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Afar,Zone 01,Asayta,January,2005,NaN,NaN,11.5,8.0 - 14.9,1.97,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
crop_df.head()

,Season,Year,Country,Region,Zone,Crop,Category,Number of holders,Area in hectare,Production in quintal,Yield (qt / ha)
0,Meher,2013/2014,Ethiopia,Afar,Zone 03,Maize,Cereals,3419,NaN,NaN,NaN
1,Meher,2014/2015,Ethiopia,Afar,Zone 03,Maize,Cereals,4000,1480.92,NaN,NaN
2,Meher,2014/2015,Ethiopia,Afar,Zone 03,Sorghum,Cereals,2542,1244.23,NaN,NaN
3,Meher,2010/2011,Ethiopia,Afar,Zone 03,Maize,Cereals,5935,NaN,NaN,NaN
4,Meher,2015/2016,Ethiopia,Afar,Zone 03,Teff,Cereals,1550,NaN,NaN,NaN


In [10]:
chirp_df.head()

,Admin0,Admin2,Year,Month,CHIRPS(mm)
0,Ethiopia,arsi,2002,Jan,20.795020
1,Ethiopia,arsi,2002,Feb,17.654432
2,Ethiopia,arsi,2002,Mar,78.814980
3,Ethiopia,arsi,2002,Apr,79.970240
4,Ethiopia,arsi,2002,May,70.014500


In [12]:
ndvi_df.head()

,Admin0,Admin2,Year,Month,NDVI
0,Ethiopia,segen,2002,Jan,0.377084
1,Ethiopia,segen,2002,Feb,0.357881
2,Ethiopia,segen,2002,Mar,0.354559
3,Ethiopia,segen,2002,Apr,0.493129
4,Ethiopia,segen,2002,May,0.539275


In [85]:
def column_standardization(df, col_name, typo_dictionary):
    df[col_name]=df[col_name].str.strip()
    df.replace({col_name: typo_dictionary}, inplace=True)
    df[col_name]=df[col_name].str.lower()
    
    return df

In [86]:
pop_df_std=column_standardization(pop_df_melt, 'admin2', typo_dict)
ndvi_df_std=column_standardization(ndvi_df, 'Admin2', typo_dict)
crop_df_std=column_standardization(crop_df, 'Zone', typo_dict)

In [87]:
maln_df['Month in survey']=maln_df['Month in survey'].str.strip()
maln_df.dropna(subset=['Month in survey'],  inplace=True)

In [88]:
import datetime
month_abb=[]
for i, row in maln_df.iterrows():
    row['Month in survey']
    month_abb.append(datetime.datetime.strptime(row['Month in survey'],'%B').strftime('%b'))

In [89]:
maln_df['Month']=month_abb
maln_df.rename(columns={'year': 'Year'}, inplace=True)

In [90]:
maln_df_std=column_standardization(maln_df, 'Admin2', typo_dict)

In [91]:

maln_df_std.replace({'Year': {'2007-2008': 2008, '2008-2009': 2009, '2010-2011': 2011}}, inplace=True)
maln_df_std['Year']=maln_df_std['Year'].astype(int)

In [92]:
pop_df_std.rename(columns={'admin2': 'Admin2', 'admin1': 'Admin1', 'admin0':'Admin0', 'YEAR':"Year"}, inplace=True)

In [93]:
pop_df_std['POP']=pop_df_std['POP'].str.replace(',','').astype(int)

In [94]:
pop_df_std['Year']=pop_df_std['Year'].astype(int)

#### introduce lag to chirps and ndvi


In [95]:
ndvi_df_std['NDVI_lag1'] = ndvi_df_std["NDVI"].shift(1)
chirp_df_std['CHIRPS(mm)_lag3'] =chirp_df_std['CHIRPS(mm)'].shift(3)


In [96]:
chirp_df_std.head()

,Admin0,Admin2,Year,Month,CHIRPS(mm),CHIRPS(mm)_lag3
0,Ethiopia,arsi,2002,Jan,20.795020,NaN
1,Ethiopia,arsi,2002,Feb,17.654432,NaN
2,Ethiopia,arsi,2002,Mar,78.814980,NaN
3,Ethiopia,arsi,2002,Apr,79.970240,20.795020
4,Ethiopia,arsi,2002,May,70.014500,17.654432


# crop production data process - group the cereal production mass by the zone(admin2) and year

In [98]:

crop_df_std.rename(columns={ 'Country':'Admin0', 'Region': 'Admin1',
                            'Zone': 'Admin2', ' Category': 'Category'}, inplace=True)
crop_df_std.replace({'Year': {'2004/2005':2005,
                             '2005/2006': 2006, 
                              '2006/2007': 2007, 
                              '2007/2008': 2008,
                              '2008/2009':2009, 
                              '2010/2011':2011,
                              '2011/2012':2012,
                              '2012/2013':2013, 
                              '2013/2014':2014,
                              '2014/2015':2015, 
                              '2015/2016':2016},
                    }, inplace=True)
crop_df_std.dropna(subset=["Production in quintal"], inplace=True)

In [99]:
crop_df_std.head()

,Season,Year,Admin0,Admin1,Admin2,Crop,Category,Number of holders,Area in hectare,Production in quintal,Yield (qt / ha)
5,Meher,2016,Ethiopia,Afar,zone 3,Maize,Cereals,5133,2048.49,49270.18,24.05
6,Meher,2015,Ethiopia,Afar,zone 1,Maize,Cereals,4576,2861.77,102920.76,35.96
13,Meher,2013,Ethiopia,Afar,zone 1,Maize,Cereals,5431,3332.55,188482.63,56.56
17,Meher,2012,Ethiopia,Afar,zone 1,Maize,Cereals,4775,NaN,140498.06,NaN
24,Meher,2016,Ethiopia,Afar,zone 1,Maize,Cereals,490,282.37,7126.61,25.24


In [100]:
#convert to tonnes
crop_df_std['Production in tonnes'] = pd.to_numeric(crop_df_std["Production in quintal"], errors='coerce')*0.05395813

In [101]:
crop_sum_df=crop_df_std.groupby(['Admin1','Admin2', 'Year','Category']).sum()["Production in tonnes"].reset_index()
crop_cereal_df=crop_sum_df.loc[crop_sum_df['Category']=='Cereals']


# CPI dataset reformat

In [102]:
cpi_df_std.head()

,Year,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec
0,2007,100.4,102.3,104.3,105.8,106.3,107.4,109.6,112.7,115.3,117.1,117.2,118.4
1,2008,120.2,125.7,135.2,137.2,147.8,166.7,179.9,182.3,184.1,182.0,175.1,165.0
2,2009,165.6,167.1,167.2,169.2,168.8,171.2,173.3,175.1,176.6,175.3,176.1,176.7
3,2010,178.2,178.9,179.6,180.7,181.2,183.7,183.1,184.3,189.9,194.0,194.1,202.4
4,2011,209.7,208.7,224.5,233.9,244.1,253.6,254.9,259.2,266.0,271.1,270.4,274.9


In [103]:
cpi_month_cols= [col for col in cpi_df_std.columns[1:].tolist()]
cpi_df_melt=pd.melt(cpi_df_std, id_vars=['Year'],
   value_vars=cpi_month_cols, var_name='Month',value_name='CPI')

# consumption expenditure data

In [104]:
consumption_df.head()

,region,food,non_food,total,currency,year,PPP conversion factor,PPP_food_Value,PPP_nonfood_Value,PPP_total_Value,expenditure_daily
0,Tigray,2115.0,2803.0,4917,Birr,2010/11,4.21,502.375297,665.795724,1167.933492,3.199818
1,Afar,2545.0,1927.0,4472,Birr,2010/11,4.21,604.513064,457.719715,1062.232779,2.910227
2,Amhara,2018.0,2651.0,4668,Birr,2010/11,4.21,479.334917,629.691211,1108.788599,3.037777
3,Oromiya,2135.0,2436.0,4570,Birr,2010/11,4.21,507.125891,578.622328,1085.510689,2.974002
4,Somali,2416.0,1863.0,4279,Birr,2010/11,4.21,573.871734,442.517815,1016.389549,2.784629


In [105]:
def expenditure_merge(expend_df, main_df):
    """
    This function integrates PPP consumption expenditure (daily) into the main data frame.
    expend_df: imported from the consumption expenditure csv (column 'expenditure_daily')
    main_df: the main dataframe used for training and prediction 
    Note: due to the limited granularity of the expenditure data, for years before 2008, expenditure consumption
    from 2005/2006 will be used. For year after 2009,  expenditure consumption
    from 2010/2011 will be used.
    """
    exp_val=[]
    for i, row in main_df.iterrows():
        if row['Year']<2008:  
            expend_daily=expend_df.loc[(expend_df['region']==row['Admin1']) & (consumption_df['year'] =='2005/06')]['expenditure_daily']
            if not expend_daily.empty:
                exp_val.append(expend_daily.values[0])
            else:
                exp_val.append(np.nan)
        
        elif row['Year'] >=2008:
            expend_daily=expend_df.loc[(expend_df['region']==row['Admin1']) & (consumption_df['year'] =='2010/11')]['expenditure_daily']
            if not expend_daily.empty:
                exp_val.append(expend_daily.values[0])
            else:
                exp_val.append(np.nan)            
        else:
            exp_val.append(np.nan)
    
    main_df['expenditure']=exp_val
    return main_df

In [106]:
maln_df_std2=expenditure_merge(consumption_df, maln_df_std)

# open files saved previously
maln_df_std=pd.read_csv('Ethiopia/cleaned_data/maln_ds_std.csv')
ndvi_df_std=pd.read_csv('Ethiopia/cleaned_data/ndvi_df_std.csv')
chirp_df_std=pd.read_csv('Ethiopia/cleaned_data/chirp_df_std.csv')
pop_df_std=pd.read_csv('Ethiopia/cleaned_data/pop_df_std.csv')
crop_df_std=pd.read_csv('Ethiopia/cleaned_data/crop_df_std.csv')
cpi_df_std=pd.read_csv('Ethiopia/cpi/ethiopia-cpi-2007-2019-country-level.csv')
cereal_crop_df=pd.read_csv('Ethiopia/cleaned_data/cereal_crop_eth.csv')

# merge the data frames

In [107]:
cereal_crop_df=pd.read_csv('Ethiopia/cleaned_data/cereal_crop_eth.csv')

In [108]:
cereal_crop_df['Year'].value_counts()

2009    68
2008    68
2011    65
2016    64
2012    64
2013    63
2015    62
2014    62
2007    60
2006    59
2005    58
Name: Year, dtype: int64

In [109]:
pop_df_std.dtypes

Admin0    object
Admin1    object
Admin2    object
POP        int64
Year       int64
dtype: object

In [110]:
maln_df_select=maln_df_std2[['Admin1','Admin2', 'Month','Year', 'GAM  Z-scores','SAM Z-scores', 'expenditure']]
merge_df1=maln_df_select.merge(chirp_df_std[['Admin2', 'Year','Month','CHIRPS(mm)_lag3']] , on=['Admin2', 'Month', 'Year'])
merge_df2=merge_df1.merge(ndvi_df_std[['Admin2', 'Year','Month','NDVI_lag1']],on=['Admin2', 'Month', 'Year'])
merge_df3=merge_df2.merge(pop_df_std[['Admin2', 'Admin0','POP','Year']], on=['Admin2', 'Year'] )
print(merge_df1.shape)
print(merge_df2.shape)
print(merge_df3.shape)

merge_df3.head(n=20)


(953, 8)
(740, 9)
(797, 11)


,Admin1,Admin2,Month,Year,GAM Z-scores,SAM Z-scores,expenditure,CHIRPS(mm)_lag3,NDVI_lag1,Admin0,POP
0,Amhara,awi,Feb,2006,7.7,0.800,2.011905,15.507689,0.367416,Ethiopia,1025925
1,Amhara,awi,Apr,2004,0.136,0.022,2.011905,5.382840,0.306070,Ethiopia,979719
2,Amhara,east gojam,Feb,2003,12.6,1.300,2.011905,7.496276,0.315513,Ethiopia,2100703
3,Amhara,north gondar,May,2018,6.8,1.100,3.037777,5.451314,0.250051,Ethiopia,4351763
4,Amhara,north gondar,Apr,2010,16.7,1.300,3.037777,3.633791,0.232076,Ethiopia,3434023
5,Amhara,north gondar,May,2010,9.1,1.000,3.037777,2.831052,0.242129,Ethiopia,3434023
6,Amhara,north gondar,Nov,2010,11.5,1.300,3.037777,304.255740,0.643141,Ethiopia,3434023
7,Amhara,north gondar,Apr,2008,5.9,0.400,3.037777,3.679500,0.248937,Ethiopia,3228656
8,Amhara,north gondar,May,2008,7.2,0.500,3.037777,2.614258,0.277260,Ethiopia,3228656
9,Amhara,north gondar,Nov,2004,0.293,0.068,2.011905,238.907500,0.649866,Ethiopia,2854734


In [111]:
cereal_crop_df['Year'].unique()

array([2005, 2006, 2007, 2008, 2009, 2012, 2013, 2015, 2016, 2011, 2014])

In [113]:
merge_df4=merge_df3.merge(cpi_df_melt, on=['Year', 'Month'] )
merge_df5=merge_df4.merge(cereal_crop_df, on=['Year', 'Admin2'] )
#remove the comma
merge_df5['Population']=merge_df5['POP']
merge_df5['crop_per_capita']=merge_df5['Production in tonnes']/merge_df5['Population']
print(merge_df4.shape)
print(merge_df5.shape)

merge_df5.head(n=20)


(347, 12)
(233, 17)


,Admin1_x,Admin2,Month,Year,GAM Z-scores,SAM Z-scores,expenditure,CHIRPS(mm)_lag3,NDVI_lag1,Admin0,POP,CPI,Admin1_y,Category,Production in tonnes,Population,crop_per_capita
0,Amhara,north gondar,Apr,2008,5.9,0.4,3.037777,3.679500,0.248937,Ethiopia,3228656,137.2,Amhara,Cereals,371616.002434,3228656,0.115099
1,Amhara,north gondar,May,2008,7.2,0.5,3.037777,2.614258,0.277260,Ethiopia,3228656,147.8,Amhara,Cereals,371616.002434,3228656,0.115099
2,Oromiya,west hararge,Apr,2008,7.7,0.3,2.974002,10.110549,0.272964,Ethiopia,2066848,137.2,Oromiya,Cereals,219206.327620,2066848,0.106058
3,Oromiya,west hararge,Apr,2008,6.9,0.8,2.974002,10.110549,0.272964,Ethiopia,2066848,137.2,Oromiya,Cereals,219206.327620,2066848,0.106058
4,Oromiya,west hararge,Sep,2008,12.4,2.1,2.974002,34.903670,0.532026,Ethiopia,2066848,184.1,Oromiya,Cereals,219206.327620,2066848,0.106058
5,SNNP,sidama,Apr,2008,19.5,3.2,2.901116,17.451496,0.416624,Ethiopia,3276642,137.2,SNNP,Cereals,62437.308575,3276642,0.019055
6,SNNP,sidama,May,2008,11.6,1.0,2.901116,16.200972,0.473905,Ethiopia,3276642,147.8,SNNP,Cereals,62437.308575,3276642,0.019055
7,SNNP,sidama,Jul,2008,7.7,0.3,2.901116,118.086105,0.576025,Ethiopia,3276642,179.9,SNNP,Cereals,62437.308575,3276642,0.019055
8,SNNP,sidama,Dec,2008,6.3,0.3,2.901116,146.383990,0.720334,Ethiopia,3276642,165.0,SNNP,Cereals,62437.308575,3276642,0.019055
9,SNNP,sidama,Dec,2008,8.1,0.6,2.901116,146.383990,0.720334,Ethiopia,3276642,165.0,SNNP,Cereals,62437.308575,3276642,0.019055


In [114]:
merge_df5.rename(columns={'GAM  Z-scores': 'gam_rate','SAM Z-scores': 'sam_rate', 
                         'expenditure': 'med_exp', 'Admin1_x':'Admin1'}, inplace=True)
merge_df5['gam_rate']=merge_df5['gam_rate'].astype(float)/100
merge_df5['sam_rate']=merge_df5['sam_rate'].astype(float)/100


rocess - group the cereal production mass by zone(admin2) and year

In [115]:
merge_df5.head()

,Admin1,Admin2,Month,Year,gam_rate,sam_rate,med_exp,CHIRPS(mm)_lag3,NDVI_lag1,Admin0,POP,CPI,Admin1_y,Category,Production in tonnes,Population,crop_per_capita
0,Amhara,north gondar,Apr,2008,0.059,0.004,3.037777,3.679500,0.248937,Ethiopia,3228656,137.2,Amhara,Cereals,371616.002434,3228656,0.115099
1,Amhara,north gondar,May,2008,0.072,0.005,3.037777,2.614258,0.277260,Ethiopia,3228656,147.8,Amhara,Cereals,371616.002434,3228656,0.115099
2,Oromiya,west hararge,Apr,2008,0.077,0.003,2.974002,10.110549,0.272964,Ethiopia,2066848,137.2,Oromiya,Cereals,219206.327620,2066848,0.106058
3,Oromiya,west hararge,Apr,2008,0.069,0.008,2.974002,10.110549,0.272964,Ethiopia,2066848,137.2,Oromiya,Cereals,219206.327620,2066848,0.106058
4,Oromiya,west hararge,Sep,2008,0.124,0.021,2.974002,34.903670,0.532026,Ethiopia,2066848,184.1,Oromiya,Cereals,219206.327620,2066848,0.106058


In [117]:
select_cols=["Admin1", 
             'Admin2',
            "gam_rate",
            "sam_rate",
            "Year",
            "NDVI_lag1",
            "Population",
            "CPI",
            "crop_per_capita",
            "CHIRPS(mm)_lag3",
            "med_exp",
             'Month']


In [120]:
merge_df5[select_cols].to_csv('Ethiopia/eth_maln_dataset.csv', index=False)

In [119]:
# save all cleaned processed dataframes
pop_df_std.to_csv('Ethiopia/cleaned_data/pop_df_std.csv', index=False) 
ndvi_df_std.to_csv('Ethiopia/cleaned_data/ndvi_df_std.csv', index=False)
chirp_df.to_csv('Ethiopia/cleaned_data/chirp_df_std.csv', index=False)
crop_cereal_df.to_csv('Ethiopia/cleaned_data/cereal_crop_eth.csv', index=False)
maln_df_std.to_csv("Ethiopia/cleaned_data/maln_ds_std.csv", index=False)
cpi_df_melt.to_csv('Ethiopia/cleaned_data/eth_cpi_2007-2019.csv', index=False)

In [121]:
len(merge_df5)

233